# Remarques :
2. branching on node / edge ? node (sub problem)
4. first column : artificial (calculate columns et solve BP)

# To do :
1. bornes dans colonnes génération (ok) et B&Price 
2. enum_smallcycles length = 2,4... = 3 marche pas, MIP problème?
3. global solution line 40 process_node

In [1]:
using JuMP 
using Gurobi
using DelimitedFiles
using NBInclude
using GraphPlot 
using LightGraphs
using MetaGraphs
const GUROBI_ENV = Gurobi.Env()
const ϵ = 0.00001;

Academic license - for non-commercial use only - expires 2021-06-25


In [152]:
# to save graph
using Cairo, Compose

┌ Info: Precompiling Cairo [159f3aea-2a34-519c-b102-8c37f9878175]
└ @ Base loading.jl:1278


LoadError: ArgumentError: Package Compose not found in current path:
- Run `import Pkg; Pkg.add("Compose")` to install the Compose package.


In [6]:
@nbinclude("typedef.ipynb")
@nbinclude("data.ipynb")
@nbinclude("MIP.ipynb")
@nbinclude("master.ipynb")
@nbinclude("subproblem.ipynb")
@nbinclude("branch_and_price.ipynb")
@nbinclude("node.ipynb");

# I - Read data

In [3]:
data_folder = string(@__DIR__,"/../data")
instance = "MD-00001-00000191"
wmd_file = joinpath(data_folder, join([instance, ".wmd"]))
dat_file = joinpath(data_folder, join([instance, ".dat"]))
global pool = kep_pool(wmd_file, dat_file)
global weight = [get_prop(pool, arc, :weight) for arc in edges(pool)]
global e = collect(edges(pool))
global L = 2;

In [ ]:
gplot(pool,nodelabel = 1:nv(pool))

In [155]:
# sauvegarde graph
draw(PNG("KEP_10.png", 16cm, 16cm), gplot(pool,nodelabel = 1:nv(pool)))

# II - Solve by MIP
Implement and solve an integer programming model for KEP: (**formualtion to change**)
\begin{align*}
\min\;\;\; &\sum_{i\in N}\sum_{j\in M} p_{ij} x_{ij}\\
\mbox{s.t.}\;\;\; 	&\sum_{j\in M} x_{ij}=1 &\forall i\in N\\
&\sum_{i\in N} t_{ij}x_{ij} \leq t^{\rm max}_{j} &\forall j\in M\\
&x\in \{0,1\}^{N\times M}.
\end{align*}


In [4]:
result = @timed solve_MIP(pool,L)

Academic license - for non-commercial use only - expires 2021-06-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 512 rows, 555738 columns and 1659218 nonzeros
Model fingerprint: 0x50731169
Variable types: 0 continuous, 555738 integer (555738 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 321.0000000
Presolve removed 3 rows and 30948 columns (presolve time = 7s) ...
Presolve removed 3 rows and 30948 columns
Presolve time: 6.96s
Presolved: 509 rows, 524790 columns, 1566366 nonzeros
Found heuristic solution: objective 322.0000000
Variable types: 0 continuous, 524790 integer (524790 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4708000e+04   1.416700e+04   0.000000e+00

(value = (351.0, [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]), time = 103.516315901, bytes = 4756488129, gctime = 2.954328193, gcstats = Base.GC_Diff(4756488129, 1858, 31, 69427741, 12299, 0, 2954328193, 37, 3))

In [29]:
enum_smallcycles(pool,3)

(Any[[1, 6], [1, 6, 3, 8], [3, 8]], Any[2, 4, 2])

In [21]:
for arc in edges(pool)
    println("There is an arc going from node $(arc.src) to $(arc.dst). It is: ",arc)
end

There is an arc going from node 1 to 3. It is: Edge 1 => 3
There is an arc going from node 1 to 16. It is: Edge 1 => 16
There is an arc going from node 2 to 4. It is: Edge 2 => 4
There is an arc going from node 2 to 14. It is: Edge 2 => 14
There is an arc going from node 3 to 4. It is: Edge 3 => 4
There is an arc going from node 3 to 14. It is: Edge 3 => 14
There is an arc going from node 4 to 6. It is: Edge 4 => 6
There is an arc going from node 4 to 13. It is: Edge 4 => 13
There is an arc going from node 4 to 16. It is: Edge 4 => 16
There is an arc going from node 5 to 1. It is: Edge 5 => 1
There is an arc going from node 5 to 2. It is: Edge 5 => 2
There is an arc going from node 5 to 4. It is: Edge 5 => 4
There is an arc going from node 5 to 6. It is: Edge 5 => 6
There is an arc going from node 5 to 8. It is: Edge 5 => 8
There is an arc going from node 5 to 10. It is: Edge 5 => 10
There is an arc going from node 5 to 11. It is: Edge 5 => 11
There is an arc going from node 5 to 13. I

# III - B&Price

In [ ]:
result = @timed solve_BP()

Current list of nodes to be processed:[1]
 Processing node 1 
Variables set to zero are:Tuple[]
Variables set to one are:Tuple[]
 Feasible solution with value 1.0 found 
 Pattern with items [1, 500] added 
 Feasible solution with value 2.0 found 
 Pattern with items [141, 502] added 
 Feasible solution with value 4.0 found 
 Pattern with items [141, 502] added 
 Feasible solution with value 4.0 found 
 Pattern with items [500, 512] added 
 Feasible solution with value 4.0 found 
 Pattern with items [502, 510] added 
 Feasible solution with value 4.0 found 
 Pattern with items [252, 481] added 
 Feasible solution with value 6.0 found 
 Pattern with items [252, 481] added 
 Feasible solution with value 6.0 found 
 Pattern with items [503, 511] added 
 Feasible solution with value 8.0 found 
 Pattern with items [503, 511] added 
 Feasible solution with value 8.0 found 
 Pattern with items [243, 481] added 
 Feasible solution with value 8.0 found 
 Pattern with items [37, 509] added 
 Feas

In [28]:
solution

3-element Array{Any,1}:
 0.0
 0.0
 1.0

## Problèmes

π négative ?--> prendre valeur abs     
1 3 6 8 ajoutés +s fois par sous problème ? Pb de code / article 1 section 5.2.3  
solution toujour entière par génération de colonne ?   
UB = 4?       

# Brouillon pour débeuger

In [79]:
calculate_ysol(solution,node_pool)

16-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

# A la main

In [193]:
global column_pool = Array{Array{Int,1},1}()
# this is a 2-dimensional array
# column_pool[i] is a pattern  (1-dimensional array = vector)
# pattern vectors are  such that:
#   -the first element is the objective value contribution
#   -the second to n+1 st elements are 1 or 0 depending on whether nth pair is included in the pattern or not


push!(column_pool,vcat(2,[0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0])) # push an artificial pattern 
#push!(column_pool,zeros(Int,nv(pool)+1))    # push an empty pattern 
push!(column_pool,vcat(2,[0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0]))


# set the global lower and upper bound to +,- infinity
global UB = Inf  # If we have a heuristic solution its value can be used   
global LB = -Inf # If we have an initial relaxation bound it can be used 

global Queue = Vector{Int}() # the Queue of nodes to process is an array of integers
global tree = Vector{TreeNode}() # the branch and price tree is an array of TreeNode objects

# the branch and price tree is initialized with the root node
# the root node is its own parent, it has no children (at the moment), its LB is -infinity, 
# there are no variables set to 0 or 1 at the root node
push!(tree,TreeNode(0,[],Inf,[],[]))   
push!(Queue,1)  # the indices of nodes to be processed are pushed to the Queue


1-element Array{Int64,1}:
 1

In [194]:
current = Queue[end]

1

## génération de colonne

In [195]:

global nodestobedeleted = []
    
# Calculate columns that are compatible with the branching constraints
# Function calculate_columns TO COMPLETE (see above)
node_pool = calculate_columns(1)

2-element Array{Array{Int64,1},1}:
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
 [2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

In [196]:
choosecerc,z,mastermodel = node_master(node_pool)

(ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape}[choosecerc[1] : 0 <= 1.0, choosecerc[2] : 0 <= 1.0, choosecerc[3] : z_2 <= 1.0, choosecerc[4] : 0 <= 1.0, choosecerc[5] : 0 <= 1.0, choosecerc[6] : 0 <= 1.0, choosecerc[7] : 0 <= 1.0, choosecerc[8] : 0 <= 1.0, choosecerc[9] : 0 <= 1.0, choosecerc[10] : 0 <= 1.0, choosecerc[11] : 0 <= 1.0, choosecerc[12] : 0 <= 1.0, choosecerc[13] : 0 <= 1.0, choosecerc[14] : z_1 + z_2 <= 1.0, choosecerc[15] : z_1 <= 1.0, choosecerc[16] : 0 <= 1.0], VariableRef[z_1, z_2], A JuMP Model
Maximization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 16 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer sta

In [232]:
mastermodel

A JuMP Model
Maximization problem with:
Variables: 3
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 16 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: choosecerc

In [233]:
choosecerc

16-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 choosecerc[1] : 0 <= 1.0
 choosecerc[2] : 0 <= 1.0
 choosecerc[3] : z_2 <= 1.0
 choosecerc[4] : z_3 <= 1.0
 choosecerc[5] : 0 <= 1.0
 choosecerc[6] : 0 <= 1.0
 choosecerc[7] : 0 <= 1.0
 choosecerc[8] : 0 <= 1.0
 choosecerc[9] : 0 <= 1.0
 choosecerc[10] : 0 <= 1.0
 choosecerc[11] : 0 <= 1.0
 choosecerc[12] : 0 <= 1.0
 choosecerc[13] : 0 <= 1.0
 choosecerc[14] : z_1 + z_2 <= 1.0
 choosecerc[15] : z_1 <= 1.0
 choosecerc[16] : z_3 <= 1.0

In [234]:
optimize!(mastermodel)

In [235]:
π = JuMP.dual.(choosecerc)

16-element Array{Float64,1}:
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -2.0
 -0.0
 -0.0

In [221]:
for i in 1:length(π)
    π[i]=abs(π[i])

end

In [222]:
value = JuMP.objective_value(mastermodel)

4.0

In [223]:
JuMP.dual_objective_value(mastermodel)

4.0

In [224]:
solution=[]
for m in 1:length(z)
    push!(solution,JuMP.value.(z[m]))
end
solflat = collect(Iterators.flatten(solution))

3-element Array{Float64,1}:
 0.0
 1.0
 1.0

In [225]:
nodeub = sum(π[i] for i in 1:nv(pool))
nodelb = value

4.0

In [226]:
maxobj = 0


0

In [227]:
SPobj,column,x = kep_subproblem(π,1)

(2.0, [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 1.0, 0.0  …  -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 1.0])

In [228]:
rc = SPobj

2.0

In [229]:
if rc>=maxobj maxobj=rc end

2.0

In [230]:
column_cost = sum(weight .* x)

2.0

In [231]:
push!(column_pool,vcat(column_cost,column))
# Store column in the node pool
push!(node_pool,vcat(column_cost,column))


4-element Array{Array{Int64,1},1}:
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
 [2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
 [2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [236]:
push!(z,@variable(mastermodel,lower_bound=0,upper_bound=1))

4-element Array{VariableRef,1}:
 z_1
 z_2
 z_3
 noname

In [237]:
set_name(z[end], "z_$(length(node_pool))")

In [238]:
for j in 1:nv(pool)
    set_normalized_coefficient(choosecerc[j], z[end], column[j])            
end

In [239]:
set_objective_function(mastermodel, objective_function(mastermodel) + column_cost*z[end])

In [240]:
nodeub += SPobj

4.0

In [241]:
if nodeub<=tree[1].ub tree[1].ub = nodeub end

4.0

In [242]:
2*abs((tree[1].ub-nodelb))/abs((nodelb+tree[1].ub))<ϵ || maxobj<=0

true

## Test valeur duale

In [166]:
model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
@variable(model, x,lower_bound=0,upper_bound=1)
@constraint(model, a,x >= 0.5)
@objective(model, Min, x)

optimize!(model)

@show value(x);
@show value(y);
@show objective_value(model);

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0x5cf2823d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 5e-01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  5.000000000e-01

User-callback calls 25, time in user-callback 0.00 sec


LoadError: MethodError: objects of type Float64 are not callable

In [167]:
model

A JuMP Model
Minimization problem with:
Variable: 1
Objective function type: VariableRef
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: a, x

In [168]:
JuMP.dual.(a)

1.0

In [179]:
using GLPK

In [243]:
model = Model(optimizer_with_attributes(() -> Cbc.Optimizer()))
@variable(model, x,lower_bound=0,upper_bound=1)
@constraint(model, a,x <= 0.5)
@objective(model, Max, x)

optimize!(model)

@show value(x);
@show objective_value(model);

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 0.5
After Postsolve, objective 0.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.5 - 0 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



LoadError: MethodError: objects of type Float64 are not callable

In [191]:
model

A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: VariableRef
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: a, x

In [244]:
JuMP.dual.(a)

LoadError: ArgumentError: ModelLike of type Cbc.Optimizer does not support accessing the attribute MathOptInterface.ConstraintDual(1)